In [2]:
%pip install pandas
%pip install requests
%pip install alpaca-py
%pip install pytz
%pip install pytickersymbols
%pip install pyarrow


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 44.1 MB/

In [ ]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockBarsRequest
from alpaca.data import StockHistoricalDataClient
from datetime import datetime
import pandas as pd



stock_client = StockHistoricalDataClient("PKDBAP3S6DPDWEUTLD77R256C6", "DVKUFxa3UNtu4yJuWXFWyuzGAEaqCkSgZvJvgBsectr1")

def get_historical_data():
    stock_client = StockHistoricalDataClient("PKDBAP3S6DPDWEUTLD77R256C6", "DVKUFxa3UNtu4yJuWXFWyuzGAEaqCkSgZvJvgBsectr1")
    formatted_request = StockBarsRequest(
        symbol_or_symbols=["NVDA"], # This could change right now it is a hardcoded value but in the future it could be dynamic
        start=datetime(2025, 4, 18), # This could change right now it is a hardcoded value but in the future it could be dynamic 
        end=datetime(2026,2,12), # This could change right now it is a hardcoded value but in the future it could be dynamic 
        timeframe=TimeFrame.Minute
    )
    response = stock_client.get_stock_bars(formatted_request)
    #print(response)
    return response

Stock_Data = get_historical_data()

mydf = Stock_Data.df

print(mydf.columns) # This is just to check what columns we have in the DataFrame it also is basically a test to make sure the above works
print(mydf.describe())

mydf = mydf.reset_index()
mydf = mydf.set_index("timestamp")

# This just makes it so the timezone is the same timezone as us though it could change depending on what we want to do in the future
mydf = mydf.tz_convert("US/Central")
mydf = mydf.between_time("08:30", "15:00")
mydf = mydf[mydf.index.dayofweek < 5]

market_open = pd.to_datetime("08:30").time()
market_close = pd.to_datetime("15:00").time()


full_timeline = pd.date_range(start=mydf.index.min(),end=mydf.index.max(), freq="1min", tz="US/Central")
full_timeline = full_timeline[(full_timeline.time >= market_open) & (full_timeline.time <= market_close) & (full_timeline.dayofweek < 5)]

mydf = mydf.reindex(full_timeline)

# This is basically the santization though unsure if this is how we want to handle it.
# Right now this uses the last available data point to fill in the missing values for open, high, low, and close, and fills in missing volume with 0.

# *********************** HAVE TO ASK *************************
# Do we want to handle the missing data differently? 

mydf[['open', 'high', 'low', 'close']] = mydf[['open', 'high', 'low', 'close']].ffill()
mydf['volume'] = mydf['volume'].fillna(0)


print(mydf.describe())

# ************************* SUMMARY *************************
# So all in all the DataFrame is called mydf and it has the following columns: open, high, low, close, volume. 
# It is in Central Time Zone though could change depending on what we want to do
# Also think this could all be converted to a function that takes in the symbol, start date, end date, and timezone as parameters and then returns the cleaned DataFrame. 
# Just not entirely sure what we want or how this is all going to be used in the future so I just left it as is for now.


Index(['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'], dtype='str')
                open           high            low          close  \
count  194786.000000  194786.000000  194786.000000  194786.000000   
mean      169.172977     169.241115     169.103207     169.173106   
std        24.338842      24.346925      24.331458      24.338446   
min        95.090000      95.140100      95.040000      95.080000   
25%       157.859900     157.909925     157.810000     157.858075   
50%       178.050000     178.130000     177.980000     178.050000   
75%       185.000000     185.060000     184.930000     185.000000   
max       211.985000     212.189900     211.740000     211.980000   

             volume    trade_count           vwap  
count  1.947860e+05  194786.000000  194786.000000  
mean   1.958244e+05    2221.902431     169.172825  
std    6.613354e+05    3870.025792      24.339137  
min    1.390000e+02       1.000000      95.087022  
25%    5.241000e+03     102.0000

In [6]:
from alpaca.broker import client
from alpaca.trading.requests import GetAssetsRequest

#Note: I learned only AFTER writing this, you need seperate api keys for the broker client
# Get those here: https://broker-app.alpaca.markets/dashboard

broker_client = client.BrokerClient("CKZXMIOPRAG37W7T53KW5F2YOW", "8hA9KKQhF7ZWnFdb78Pb2dqenm6hBC1NUDqdUnHvzTH")

request_params = GetAssetsRequest(
    status="active",
    asset_class="us_equity",
    exchange="NYSE"
)

nyse_assets = broker_client.get_all_assets(request_params)
nasdaq_assets = broker_client.get_all_assets(GetAssetsRequest(status="active", asset_class="us_equity", exchange="NASDAQ"))

print(f"Active NYSE assets: {len(nyse_assets)}")
print(f"Active NASDAQ assets: {len(nasdaq_assets)}")

Active NYSE assets: 2881
Active NASDAQ assets: 5322


In [7]:
def fetch_symbols_from_response(nyse_assets, nasdaq_assets):
    #This is called list comprehension
    nyse_symbols = [asset.symbol for asset in nyse_assets if asset.status == "active" and asset.tradable]
    nasdaq_symbols = [asset.symbol for asset in nasdaq_assets if asset.status == "active" and asset.tradable]
    return nyse_symbols + nasdaq_symbols
symbols_list = fetch_symbols_from_response(nyse_assets, nasdaq_assets)
print(len(symbols_list), "Count of symbols fetched from broker API")

8202 Count of symbols fetched from broker API


In [ ]:
def find_top_1000_assets(symbols_list, stock_client):
    # This function will fetch the latest daily dollar volume for each symbol and return the top 1000 by that metric.
    bars = stock_client.get_stock_bars(StockBarsRequest(
        symbol_or_symbols=symbols_list,
        timeframe=TimeFrame.Day,
        start=datetime(2026, 1, 1),
        end=datetime(2026, 2, 1)
    ))
    bars_df = bars.df
    bars_df["dollar_volume"] = bars_df["close"] * bars_df["volume"]
    # Sort symbols by dollar volume and take top 1000
    top_1000 = bars_df.groupby("symbol")["dollar_volume"].sum().nlargest(1000)
    return top_1000

top_1000_symbols = find_top_1000_assets(symbols_list, stock_client)
print("Top 1000 symbols by latest daily dollar volume:")
print(top_1000_symbols)




Top 1000 symbols by latest daily dollar volume:
symbol
QQQ     6.746866e+11
NVDA    6.141162e+11
TSLA    5.567444e+11
MSFT    3.134278e+11
MU      2.781284e+11
            ...     
CRWG    2.200888e+09
FBIN    2.197632e+09
ALC     2.195588e+09
ZION    2.194076e+09
EXEL    2.192365e+09
Name: dollar_volume, Length: 1000, dtype: float64


In [1]:
import pyarrow





def get_stock_data(ticker):
    stock_client = StockHistoricalDataClient("PKDBAP3S6DPDWEUTLD77R256C6", "DVKUFxa3UNtu4yJuWXFWyuzGAEaqCkSgZvJvgBsectr1")

    def get_historical_data(ticker):
        stock_client = StockHistoricalDataClient("PKDBAP3S6DPDWEUTLD77R256C6", "DVKUFxa3UNtu4yJuWXFWyuzGAEaqCkSgZvJvgBsectr1")
        formatted_request = StockBarsRequest(
            symbol_or_symbols=[ticker], # This could change right now it is a hardcoded value but in the future it could be dynamic
            start=datetime(2025, 4, 18), # This could change right now it is a hardcoded value but in the future it could be dynamic 
            end=datetime(2026,2,12), # This could change right now it is a hardcoded value but in the future it could be dynamic 
            timeframe=TimeFrame.Minute
        )
        response = stock_client.get_stock_bars(formatted_request)
        #print(response)
        return response

    Stock_Data = get_historical_data()

    mydf = Stock_Data.df

    print(mydf.columns) # This is just to check what columns we have in the DataFrame it also is basically a test to make sure the above works
    print(mydf.describe())

    mydf = mydf.reset_index()
    mydf = mydf.set_index("timestamp")

    mydf = mydf.tz_convert("US/Central")
    mydf = mydf.between_time("08:30", "15:00")
    mydf = mydf[mydf.index.dayofweek < 5]

    market_open = pd.to_datetime("08:30").time()
    market_close = pd.to_datetime("15:00").time()


    full_timeline = pd.date_range(start=mydf.index.min(),end=mydf.index.max(), freq="1min", tz="US/Central")
    full_timeline = full_timeline[(full_timeline.time >= market_open) & (full_timeline.time <= market_close) & (full_timeline.dayofweek < 5)]

    mydf = mydf.reindex(full_timeline)

    mydf[['open', 'high', 'low', 'close']] = mydf[['open', 'high', 'low', 'close']].ffill()
    mydf['volume'] = mydf['volume'].fillna(0)


    print(mydf.describe())
    return mydf


def convert_to_Dollar_bars(mydf):
    current_dollar_value = 0
    current_set = [] # my list of current rows I need for the dollar bar
    final_set = [] # my final dollar bar data
    
    #print(temp_mydf['Close']*temp_mydf['Volume'])
    #print()

    for index, row in temp_mydf.iterrows():
        #print(f"{row['Close']} * {row['Volume']} " )
        dollar_tot = row['Close']*row['Volume']
        current_dollar_value += dollar_tot
        #print(current_dollar_value)
        #now the goal is to populate the list with data
        current_set.append(row)
        #print(current_set)
        print()
        if current_dollar_value >= Strategist_Threshold:
            
            #new_df_dollar_bar = temp_mydf.copy()
            new_df_dollar_bar = pd.DataFrame(current_set)
            print(new_df_dollar_bar)
            open_price = new_df_dollar_bar.iloc[0]['Close']
            close_price = new_df_dollar_bar.iloc[-1]['Close']
            high_price = new_df_dollar_bar.max()['Close']
            low_price = new_df_dollar_bar.min()['Close']
            final_set.append({"Open":open_price, "High":high_price, 
                            "Low":low_price, "Close":close_price,
                            "Dollar Volume": current_dollar_value})
            current_set = []
            current_dollar_value = 0  # Note: AI recomends to do a substraction of the Strategist's data instead of setting all to 0
    final_set_df = pd.DataFrame(final_set)
    print(final_set_df)
    return final_set_df



def create_parquet_files(ticker):
    tempdf = get_stock_data(ticker)
    tempdf.to_parquet(output_filename, engine='pyarrow', compression='snappy')

    


















ModuleNotFoundError: No module named 'pyarrow'

In [5]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockBarsRequest
from alpaca.data import StockHistoricalDataClient
from datetime import datetime
import pandas as pd
import os
import pyarrow

stock_client = StockHistoricalDataClient("PKDBAP3S6DPDWEUTLD77R256C6", "DVKUFxa3UNtu4yJuWXFWyuzGAEaqCkSgZvJvgBsectr1")


from alpaca.broker import client
from alpaca.trading.requests import GetAssetsRequest

#Note: I learned only AFTER writing this, you need seperate api keys for the broker client
# Get those here: https://broker-app.alpaca.markets/dashboard

broker_client = client.BrokerClient("CKZXMIOPRAG37W7T53KW5F2YOW", "8hA9KKQhF7ZWnFdb78Pb2dqenm6hBC1NUDqdUnHvzTH")

request_params = GetAssetsRequest(
    status="active",
    asset_class="us_equity",
    exchange="NYSE"
)

nyse_assets = broker_client.get_all_assets(request_params)
nasdaq_assets = broker_client.get_all_assets(GetAssetsRequest(status="active", asset_class="us_equity", exchange="NASDAQ"))

print(f"Active NYSE assets: {len(nyse_assets)}")
print(f"Active NASDAQ assets: {len(nasdaq_assets)}")

def fetch_symbols_from_response(nyse_assets, nasdaq_assets):
    #This is called list comprehension
    nyse_symbols = [asset.symbol for asset in nyse_assets if asset.status == "active" and asset.tradable]
    nasdaq_symbols = [asset.symbol for asset in nasdaq_assets if asset.status == "active" and asset.tradable]
    return nyse_symbols + nasdaq_symbols
symbols_list = fetch_symbols_from_response(nyse_assets, nasdaq_assets)
print(len(symbols_list), "Count of symbols fetched from broker API")


# This is what I think we have to mess around with to get our top 1000 tickers 

def find_top_1000_assets(symbols_list, stock_client):
    # This function will fetch the latest daily dollar volume for each symbol and return the top 1000 by that metric.
    bars = stock_client.get_stock_bars(StockBarsRequest(
        symbol_or_symbols=symbols_list,
        timeframe=TimeFrame.Day,
        start=datetime(2026, 1, 1),
        end=datetime(2026, 2, 1)
    ))
    bars_df = bars.df
    bars_df["dollar_volume"] = bars_df["close"] * bars_df["volume"]
    # Sort symbols by dollar volume and take top 1000
    top_1000 = bars_df.groupby("symbol")["dollar_volume"].sum().nlargest(1000)
    return top_1000

top_1000_symbols = find_top_1000_assets(symbols_list, stock_client)
#print("Top 1000 symbols by latest daily dollar volume:")
#print(top_1000_symbols)






def get_stock_data(ticker):

    def get_historical_data(ticker):
        stock_client = StockHistoricalDataClient("PKDBAP3S6DPDWEUTLD77R256C6", "DVKUFxa3UNtu4yJuWXFWyuzGAEaqCkSgZvJvgBsectr1")
        formatted_request = StockBarsRequest(
            symbol_or_symbols=[ticker], # This could change right now it is a hardcoded value but in the future it could be dynamic
            start=datetime(2025, 4, 18), # This could change right now it is a hardcoded value but in the future it could be dynamic 
            end=datetime(2026,2,12), # This could change right now it is a hardcoded value but in the future it could be dynamic 
            timeframe=TimeFrame.Minute
        )
        response = stock_client.get_stock_bars(formatted_request)
        #print(response)
        return response


    mydf = get_historical_data(ticker).df

    #print(mydf.columns) # This is just to check what columns we have in the DataFrame it also is basically a test to make sure the above works
    #print(mydf.describe())

    mydf = mydf.reset_index()
    mydf = mydf.set_index("timestamp")

    mydf = mydf.tz_convert("US/Central")
    mydf = mydf.between_time("08:30", "15:00")
    mydf = mydf[mydf.index.dayofweek < 5]

    market_open = pd.to_datetime("08:30").time()
    market_close = pd.to_datetime("15:00").time()


    full_timeline = pd.date_range(start=mydf.index.min(),end=mydf.index.max(), freq="1min", tz="US/Central")
    full_timeline = full_timeline[(full_timeline.time >= market_open) & (full_timeline.time <= market_close) & (full_timeline.dayofweek < 5)]

    mydf = mydf.reindex(full_timeline)

    mydf[['open', 'high', 'low', 'close']] = mydf[['open', 'high', 'low', 'close']].ffill()
    mydf['volume'] = mydf['volume'].fillna(0)


    #print(mydf.describe())
    return mydf

def create_parquet_files(symbols_list):
    os.makedirs("data/raw", exist_ok=True)
    
    for ticker in symbols_list:
        df = get_stock_data(ticker)
        df.to_parquet(f"data/raw/{ticker}.parquet", engine='pyarrow', compression='snappy')

# This is commented out for now because it takes long to run as it builds the parquet files.
create_parquet_files(top_1000_symbols.index.tolist())


# This is just to check that we can read the parquet files we just created and that they look correct. We will delete this code later, it is just for testing purposes.

parquet_files = os.listdir("data/raw")

for file in parquet_files:
    df = pd.read_parquet(f"data/raw/{file}", engine="pyarrow")
    ticker = file.replace(".parquet", "")
    print(f"{ticker} has {len(df)} rows of data")
    print(df)

Active NYSE assets: 2883
Active NASDAQ assets: 5333
8214 Count of symbols fetched from broker API


ArrowKeyError: A type extension with name pandas.period already defined